Pytorch to fastai, Bridging the Gap

https://muellerzr.github.io/fastblog/2021/02/14/Pytorchtofastai.html

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dset_train = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transform)

dset_test = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transform)

trainloader = torch.utils.data.DataLoader(
    dset_train,
    batch_size=4,
    shuffle=True,
    num_workers=2)

testloader = torch.utils.data.DataLoader(
    dset_test,
    batch_size=4,
    shuffle=True,
    num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [2]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

In [3]:
criterion = nn.CrossEntropyLoss()

from fastai.optimizer import OptimWrapper
from torch import optim

def opt_func(params, **kwargs): return OptimWrapper(optim.SGD(params, lr=0.001))

In [5]:
from fastai.data.core import DataLoaders

dls = DataLoaders(trainloader, testloader)

from fastai.learner import Learner

from fastai.callback.progress import ProgressCallback

from fastai.callback.data import CudaCallback

In [7]:
learn = Learner(
    dls,
    net,
    loss_func=criterion,
    opt_func=opt_func,
    cbs=[CudaCallback])

In [8]:
learn.fit(2)

torch.save(learn.model.state_dict(), './cifar_net.pth')

In [9]:
print("Hello world")

Hello world
